In [1]:
%env CUDA_VISIBLE_DEVICES=2

env: CUDA_VISIBLE_DEVICES=2


In [2]:
# pip install accelerate
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b")
model = AutoModelForCausalLM.from_pretrained(
    "google/gemma-2-2b",
    device_map="auto",
)

input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids, max_new_tokens=32)
print(tokenizer.decode(outputs[0]))


/gpfs01/berens/user/jdoehl/GitHub/LoRA-Embedding-Layer/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 3/3 [00:13<00:00,  4.55s/it]
/gpfs01/berens/user/jdoehl/GitHub/LoRA-Embedding-Layer/.venv/lib/python3.12/site-packages/torch/_inductor/compile_fx.py:236: UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
  warnings.warn(


<bos>Write me a poem about Machine Learning.

[Answer 1]

<strong>Machine Learning</strong>

Machine learning is a field of study that is very much like a machine.

It is a field


In [33]:

input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

outputs = model.generate(**input_ids, max_new_tokens=32)
print(tokenizer.decode(outputs[0]))

<bos>Write me a poem about Machine Learning.

[Answer 1]

<strong>Machine Learning</strong>

Machine learning is a field of study that is very much like a machine.

It is a field


In [9]:
torch.save(model.model.embed_tokens.state_dict(), "gemma-2-2b_embedding-layer.pth")

In [38]:
embed_layer = torch.load("gemma-2-2b_embedding-layer.pth")["weight"]

In [39]:
embed_layer.shape

torch.Size([256000, 2304])

In [20]:
tokenizer.vocab

{'▁Gonna': 111830,
 '▁Merkmale': 151292,
 '▁hro': 214626,
 '▁REDUCTION': 217895,
 '▁Objekte': 149772,
 '▁stent': 178200,
 '#(': 181536,
 '▁infancia': 106198,
 'ceps': 101857,
 '▁prévention': 148035,
 '▁Holder': 54217,
 'hild': 203104,
 '▁sorento': 221657,
 'adra': 153232,
 '▁المك': 174414,
 '祓': 244058,
 'Hol': 34090,
 '▁teneur': 231212,
 'بخ': 130656,
 'ativität': 208967,
 '\uf036': 248055,
 '▁museums': 52054,
 '▁income': 8316,
 'Rb': 98646,
 '▁鳥': 101968,
 '▁virtud': 127105,
 '▁probiotics': 168612,
 '▁лишь': 53048,
 'acté': 27936,
 '▁redeeming': 180899,
 '_);': 31736,
 '▁доро': 31837,
 '▁Branchen': 232875,
 'cą': 65378,
 'perrt': 218635,
 '▁tchèque': 136582,
 '▁pomoci': 232033,
 '▁mis': 2586,
 '▁Egon': 231365,
 '%"),': 79921,
 'atka': 165889,
 'uland': 164392,
 'timeter': 183903,
 '▁greu': 193390,
 'xtype': 82006,
 '▁stickied': 232680,
 '▁presentan': 93315,
 'यह': 166878,
 '▁tomorrow': 14610,
 '▁cifras': 114289,
 '▁Endpoint': 109169,
 '▁casio': 90225,
 '▁fraud': 19367,
 '▁εύ': 227830

In [43]:
normalized_embed_layer = embed_layer / embed_layer.norm(dim=-1, keepdim=True)
nld_embed_layer = normalized_embed_layer

In [86]:
input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt")
print(tokenizer.convert_ids_to_tokens(input_ids["input_ids"][0]))

['<bos>', 'Write', '▁me', '▁a', '▁poem', '▁about', '▁Machine', '▁Learning', '.']


In [88]:
cur_id = input_ids["input_ids"][0,8]
cos_sim = nld_embed_layer @ nld_embed_layer[cur_id]
cos_sim_sort = cos_sim.argsort()
k = 20
top_k_ids = cos_sim_sort[-k:]
top_k_sim = cos_sim[top_k_ids]
print(top_k_sim)
print(tokenizer.convert_ids_to_tokens([cur_id]))
print(tokenizer.convert_ids_to_tokens(top_k_ids))

tensor([0.6114, 0.6120, 0.6134, 0.6269, 0.6346, 0.6388, 0.6392, 0.6469, 0.6516,
        0.6536, 0.6552, 0.6569, 0.6727, 0.6750, 0.6813, 0.6999, 0.7191, 0.7203,
        0.7806, 1.0000], device='cuda:0')
['.']
['_', '?', '1', '/', '▁and', '▁.', '▁in', ')', '▁-', '(', '\n\n', '!', '\n', '▁', '-', '▁(', ';', ':', ',', '.']
